# This notebook is used to test the ANNABELL model on the SQuAD dataset.


In [11]:
from dataset_processing import any_word_match, embedding_for_sentence, \
    cosine_distance, ids_questions_answers_from_log_file
import os
import platform
import datetime
import pandas as pd
from tqdm import tqdm

In [12]:
experiment_number = 12
operating_system = platform.system()
if operating_system == 'Windows':
    raise Exception("not used on Windows yet")
elif operating_system == 'Linux':
    base_directory = "/home/chris/gdrive/work/annabell"
elif operating_system == 'Darwin':  #macOS
    base_directory = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/"
else:
    raise Exception("unsupported OS")

timestamp = datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")

test_input_dir = os.path.join(base_directory, "experiments", str(experiment_number), "testing")
test_log_dir = os.path.join(base_directory, "experiments", str(experiment_number), "logs")
data_dir = os.path.join(base_directory, "experiments/data")
test_results_dir = os.path.join(base_directory, "experiments", str(experiment_number), "results")
if not os.path.exists(test_results_dir):
    os.makedirs(test_results_dir)

test_log_filename = "logfile_nyc_squad_testing_commands_2025-10-15_08-31-02.txt"
test_input_filename = "nyc_squad_testing_commands_20251014_063732.txt"
dataset_filename = "nyc_squad_with_pretraining_commands_20251014_063732.jsonl"

test_log_filepath = os.path.join(test_log_dir, test_log_filename)
test_input_filepath = os.path.join(test_input_dir, test_input_filename)
dataset_filepath = os.path.join(data_dir, dataset_filename)

test_answer_summary_filename = "test_nyc_answer_summary" + timestamp + ".tsv"
tsv_results_filepath = os.path.join(test_results_dir, test_answer_summary_filename)

ids_questions_answers = ids_questions_answers_from_log_file(test_log_filepath)
for index, each_tuple in enumerate(ids_questions_answers):
    if each_tuple[-1] == "" or each_tuple[-1] == None:
        ids_questions_answers[index] = (each_tuple[0], each_tuple[1], "NO ANSWER GIVEN")

print("length of log file questions and answers: " + str(len(ids_questions_answers)))
with open(test_input_filepath, 'r') as test_input_file:
    test_input_lines = test_input_file.readlines()
response_formatted_df = pd.read_json(dataset_filepath, lines=True)
#add the test questions to the dataframe
questions_not_found = []
for the_id, question, answer, in ids_questions_answers:
    if the_id in response_formatted_df["id"].values:
        response_formatted_df.loc[response_formatted_df["id"] == the_id, "test_answer"] = answer
    else:
        questions_not_found.append(question)
print(f"number of test samples not found in training data: {len(questions_not_found)}")
print("test samples not found in training data: " + str(questions_not_found[:5]) + " ...")
#drop any rows that are not in the test samples
response_formatted_df.dropna(subset=["test_answer"], inplace=True)
response_formatted_df.reset_index(inplace=True)
response_formatted_df

length of log file questions and answers: 601
number of test samples not found in training data: 0
test samples not found in training data: [] ...


,index,id,question,response_question,response_answer,response_declarative_sentence,answer,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,is_pretraining,question_category,sentence_category,created_commands,test_answer
0,0,56ce304daab44d1400b8850e,What city in the United States has the highest...,What city in the United States has the high –e...,New York,The city in the United States with the high –e...,New York,? what city in the United-States has the high ...,New-York,the city in the United-States with the high -e...,False,Wh-Subject Question,Subject-Verb-Complement (SVC),"[# ID: 56ce304daab44d1400b8850e, the city in t...",1700s
1,1,56ce304daab44d1400b8850f,In what city is the United Nations based?,In what city is the United Nations base –d?,New York,The United Nations is base –d in New York.,New York,? in what city is the United-Nations base -d,New-York,the United-Nations is base -d in New-York,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56ce304daab44d1400b8850f, the United-Na...",Exploitation number of updates >= 4000
2,2,56ce304daab44d1400b88510,What city has been called the cultural capital...,What city has been call –ed the culture –al ca...,New York,New York has been call –ed the culture –al cap...,New York,? what city has been call -ed the culture -al ...,New-York,New-York has been call -ed the culture -al cap...,False,Wh-Subject Question,Passive Construction,"[# ID: 56ce304daab44d1400b88510, New-York has ...",One-World-Trade-Center
3,3,56ce304daab44d1400b88511,What American city welcomes the largest number...,What American city welcome –s the large –st nu...,New York,New York is the American city that welcome –s ...,New York,? what American city welcome -s the large -st ...,New-York,New-York is the American city that welcome -s ...,False,Wh-Subject Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56ce304daab44d1400b88511, New-York is t...",Exploitation number of updates >= 4000
4,6,56ce3124aab44d1400b8852a,How many boroughs comprise New York City?,How many borough –s comprise New York City?,five,New York City is comprise –d of five borough –s.,five,? how many borough -s comprise New-York-City,five,New-York-City is comprise -d of five borough -s,False,Quantitative Wh-Question,Passive Construction,"[# ID: 56ce3124aab44d1400b8852a, New-York-City...",NO ANSWER GIVEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,723,56d1204617492d1400aab9fc,What federal district court has jurisdiction o...,What federal district court has jurisdiction o...,the District Court for the Eastern District of...,The District Court for the Eastern District of...,the District Court for the Eastern District of...,? what federal district court has jurisdiction...,the District-Court for the Eastern-District of...,the District-Court for the Eastern-District of...,False,Wh-Subject Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56d1204617492d1400aab9fc, the District-...",NO ANSWER GIVEN
597,724,56d1204617492d1400aab9fd,In what borough is the New York City Hall found?,In what borough is the New York City Hall found?,Manhattan,The New York City Hall is found in the borough...,Manhattan,? in what borough is the New-York-City-Hall found,Manhattan,the New-York-City-Hall is found in the borough...,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56d1204617492d1400aab9fd, the New-York-...",in
598,725,56d1218c17492d1400aaba1f,How much money in cents does New York City rec...,How much money in cent –s does New York City r...,83,New York City receive –s 83 cent –s for every ...,83,? how much money in cent -s does New-York-City...,83,New-York-City receive -s 83 cent -s for every ...,False,Quantitative Wh-Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56d1218c17492d1400aaba1f, New-York-City...",two
599,727,56d1218c17492d1400aaba21,"Each year, how much more money does New York C...","Each year, how much more money does New York C...",$ 11.4 billion,New York City give –s $ 11.4 bi

In [13]:
#generate embeddings for the test answer and the response_answer_formatted columns and compare them using cosine distance
tqdm.pandas(desc="Generating test answer embeddings")
response_formatted_df["test_answer_embedding"] = response_formatted_df["test_answer"].progress_apply(
    lambda x: embedding_for_sentence(x) if pd.notnull(x) else None)
tqdm.pandas(desc="Generating response answer embeddings")
response_formatted_df["response_answer_formatted_embedding"] = response_formatted_df[
    "response_answer_formatted"].progress_apply(
    lambda x: embedding_for_sentence(x) if pd.notnull(x) else None)

Generating response answer embeddings: 100%|██████████| 601/601 [01:02<00:00,  9.55it/s]


In [14]:
response_formatted_df["test_answer_cosine_distance"] = response_formatted_df.apply(cosine_distance, axis=1)
response_formatted_df

,index,id,question,response_question,response_answer,response_declarative_sentence,answer,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,is_pretraining,question_category,sentence_category,created_commands,test_answer,test_answer_embedding,response_answer_formatted_embedding,test_answer_cosine_distance
0,0,56ce304daab44d1400b8850e,What city in the United States has the highest...,What city in the United States has the high –e...,New York,The city in the United States with the high –e...,New York,? what city in the United-States has the high ...,New-York,the city in the United-States with the high -e...,False,Wh-Subject Question,Subject-Verb-Complement (SVC),"[# ID: 56ce304daab44d1400b8850e, the city in t...",1700s,"[-0.18142706, -0.001205836, 0.046104997, 0.007...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.208811
1,1,56ce304daab44d1400b8850f,In what city is the United Nations based?,In what city is the United Nations base –d?,New York,The United Nations is base –d in New York.,New York,? in what city is the United-Nations base -d,New-York,the United-Nations is base -d in New-York,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56ce304daab44d1400b8850f, the United-Na...",Exploitation number of updates >= 4000,"[-0.15511338, 0.028740732, 0.008751592, 0.0579...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.306653
2,2,56ce304daab44d1400b88510,What city has been called the cultural capital...,What city has been call –ed the culture –al ca...,New York,New York has been call –ed the culture –al cap...,New York,? what city has been call -ed the culture -al ...,New-York,New-York has been call -ed the culture -al cap...,False,Wh-Subject Question,Passive Construction,"[# ID: 56ce304daab44d1400b88510, New-York has ...",One-World-Trade-Center,"[-0.16647865, 0.01233991, 0.051709116, 0.00260...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.207379
3,3,56ce304daab44d1400b88511,What American city welcomes the largest number...,What American city welcome –s the large –st nu...,New York,New York is the American city that welcome –s ...,New York,? what American city welcome -s the large -st ...,New-York,New-York is the American city that welcome -s ...,False,Wh-Subject Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56ce304daab44d1400b88511, New-York is t...",Exploitation number of updates >= 4000,"[-0.15511338, 0.028740732, 0.008751592, 0.0579...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.306653
4,6,56ce3124aab44d1400b8852a,How many boroughs comprise New York City?,How many borough –s comprise New York City?,five,New York City is comprise –d of five borough –s.,five,? how many borough -s comprise New-York-City,five,New-York-City is comprise -d of five borough -s,False,Quantitative Wh-Question,Passive Construction,"[# ID: 56ce3124aab44d1400b8852a, New-York-City...",NO ANSWER GIVEN,"[-0.17646857, 0.017760377, 0.028869705, 0.0407...","[-0.21607131, 0.0066142594, 0.021365147, -0.00...",0.146334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,723,56d1204617492d1400aab9fc,What federal district court has jurisdiction o...,What federal district court has jurisdiction o...,the District Court for the Eastern District of...,The District Court for the Eastern District of...,the District Court for the Eastern District of...,? what federal district court has jurisdiction...,the District-Court for the Eastern-District of...,the District-Court for the Eastern-District of...,False,Wh-Subject Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56d1204617492d1400aab9fc, the District-...",NO ANSWER GIVEN,"[-0.17646857, 0.017760377, 0.028869705, 0.0407...","[-0.1400723, 0.008040977, 0.020870337, 0.00184...",0.356941
597,724,56d1204617492d1400aab9fd,In what borough is the New York City Hall found?,In what borough is the New York City Hall found?,Manhattan,The New York City Hall is found in the borough...,Manhattan,? in what borough is the New-Yo

In [15]:
# Get the counts for each unique value in the 'test_answer' column
test_answer_summary = response_formatted_df['test_answer'].value_counts().reset_index()
# Rename the columns for clarity
test_answer_summary.columns = ['test_answer', 'count']
# Sort the results by count in descending order
test_answer_summary.sort_values(by='count', ascending=False, inplace=True)
test_answer_summary

,test_answer,count
0,NO ANSWER GIVEN,193
1,Exploitation number of updates >= 4000,125
2,Manhattan,10
3,the,9
4,37866,8
...,...,...
83,capital,1
82,million,1
81,Charles-V,1
80,George-Washington-Bridge,1


In [16]:
#write the results dataframe to a tsv file
test_answer_summary.to_csv(tsv_results_filepath, sep="\t", index=False)
#count the number of results where the test answer is > 20 words
num_long_answers = response_formatted_df["test_answer"].apply(
    lambda x: len(x.split()) > 20 if pd.notnull(x) else False).sum()
print(f"number of test answers longer than 20 words: {num_long_answers}")
response_formatted_df["test_answer_correct"] = response_formatted_df["test_answer"] == response_formatted_df[
    "response_answer_formatted"]
number_correct = response_formatted_df["test_answer_correct"].sum()
print(f"number correct = {number_correct} out of {len(response_formatted_df)}")
percentage_correct = response_formatted_df["test_answer_correct"].mean() * 100
print(f"percentage correct = {percentage_correct} %")

response_formatted_df["test_answer_any_matching_word"] = response_formatted_df.apply(any_word_match, axis=1)
percentage_any_word_matches = response_formatted_df["test_answer_any_matching_word"].mean() * 100
number_any_word_matches = response_formatted_df["test_answer_any_matching_word"].sum()
print(f"number any word matches = {number_any_word_matches} out of {len(response_formatted_df)}")
print(f"percentage any word matches = {percentage_any_word_matches} %")

cosine_distance_threshold = 0.1
#create a dataframe with the rows where the cosine distance is less than the threshold
close_cosine_distance_df = response_formatted_df[
    response_formatted_df["test_answer_cosine_distance"] < cosine_distance_threshold]
print(f"number of rows with cosine distance less than {cosine_distance_threshold}: {len(close_cosine_distance_df)}")
print("percentage of total: " + str(len(close_cosine_distance_df) / len(response_formatted_df) * 100) + " %")

#create a dataframe with the rows where the cosine distance is less than the threshold and any matching answer is correct
close_cosine_distance_correct_df = close_cosine_distance_df[
    close_cosine_distance_df["test_answer_any_matching_word"]]
print(
    f"number of rows with cosine distance less than {cosine_distance_threshold} and any matching answer correct: {len(close_cosine_distance_correct_df)}")
print("percentage of total: " + str(len(close_cosine_distance_correct_df) / len
(response_formatted_df) * 100) + " %")

close_cosine_distance_correct_df

number of test answers longer than 20 words: 0
number correct = 19 out of 601
percentage correct = 3.1613976705490847 %
number any word matches = 24 out of 601
percentage any word matches = 3.9933444259567388 %
number of rows with cosine distance less than 0.1: 68
percentage of total: 11.314475873544092 %
number of rows with cosine distance less than 0.1 and any matching answer correct: 21
percentage of total: 3.494176372712146 %


,index,id,question,response_question,response_answer,response_declarative_sentence,answer,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,is_pretraining,question_category,sentence_category,created_commands,test_answer,test_answer_embedding,response_answer_formatted_embedding,test_answer_cosine_distance,test_answer_correct,test_answer_any_matching_word
53,66,56cfb206234ae51400d9be8f,New Netherland established a permanent Europea...,New Netherland establish –ed a permanent Europ...,1624,New Netherland establish –ed a permanent Europ...,1624,? New-Netherland establish -ed a permanent Eur...,1624,New-Netherland establish -ed a permanent Europ...,False,Wh-Adverbial Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56cfb206234ae51400d9be8f, New-Netherlan...",1624,"[-0.18510516, 0.007434764, 0.02475373, -0.0089...","[-0.18510516, 0.007434764, 0.02475373, -0.0089...",0.000000,True,True
56,70,56cedc10aab44d1400b88b1a,What was the regnal name of the Duke of York?,What was the regnal name of the Duke of York?,James II,The regnal name of the Duke of York was James II.,James II,? what was the regnal name of the Duke of York,James-II,the regnal name of the Duke of York was James-II,False,Wh-Object/Complement Question,Subject-Verb-Complement (SVC),"[# ID: 56cedc10aab44d1400b88b1a, the regnal na...",James-II,"[-0.20660716, -0.02500022, 0.020670414, 0.0114...","[-0.20660716, -0.02500022, 0.020670414, 0.0114...",0.000000,True,True
65,82,56cedd1caab44d1400b88b42,How many Lenape lived in the area in 1700?,How many Lenape live –d in the area in 1700?,200,200 Lenape live –d in the area in 1700.,200,? how many Lenape live -d in the area in 1700,200,200 Lenape live -d in the area in 1700,False,Quantitative Wh-Question,Subject-Verb-Adverbial (SVA),"[# ID: 56cedd1caab44d1400b88b42, 200 Lenape li...",200,"[-0.17841734, 0.015454305, 0.0016121272, 0.010...","[-0.17841734, 0.015454305, 0.0016121272, 0.010...",0.000000,True,True
73,92,56cede40aab44d1400b88b71,In what year was John Peter Zenger tried?,In what year was John Peter Zenger try –ed?,1735,John Peter Zenger was try –ed in 1735.,1735,? in what year was John-Peter-Zenger try -ed,1735,John-Peter-Zenger was try -ed in 1735,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56cede40aab44d1400b88b71, John-Peter-Ze...",1735,"[-0.19004162, -0.008740758, 0.054138605, 0.010...","[-0.19004162, -0.008740758, 0.054138605, 0.010...",0.000000,True,True
74,93,56cede40aab44d1400b88b72,On what island did Zenger's trial occur?,On what island did Zenger's trial occur?,Manhattan,Zenger's trial occur –ed on Manhattan.,Manhattan,? on what island did Zengers trial occur,Manhattan,Zengers trial occur -ed on Manhattan,False,Wh-Adverbial Question,Subject-Verb-Adverbial (SVA),"[# ID: 56cede40aab44d1400b88b72, Zengers trial...",Manhattan,"[-0.20684911, -0.003379386, 0.038716637, -0.00...","[-0.20684911, -0.003379386, 0.038716637, -0.00...",0.000000,True,True
75,94,56cede40aab44d1400b88b73,In what year was Columbia University chartered?,In what year was Columbia University charter –ed?,1754,Columbia University was charter –ed in 1754.,1754,? in what year was Columbia-University charter...,1754,Columbia-University was charter -ed in 1754,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56cede40aab44d1400b88b73, Columbia-Univ...",1754,"[-0.18632773, -0.0046668183, 0.04927611, 0.002...","[-0.18632773, -0.0046668183, 0.04927611, 0.002...",0.000000,True,True
77,96,56cede40aab44d1400b88b75,What was the original name of Columbia Univers...,What was the origin –al name of Columbia Unive...,King's College,The origin –al name of Columbia University was...,King's College,? what was the origin -al name of Columbia-Uni...,Kings-College,the origin -al name of Columbia-University was...,False,Wh-Object/Complement Question,Subject-Verb-Complement (SVC),"[# ID: 56cede40aab44d1400b88b75, the origin -a...",was Kings-College,"[-0.1823988, -0.017900608, 0.014207478, -0.001...","[-0.19089682,

In [17]:
correct_matches = response_formatted_df[response_formatted_df["test_answer_correct"]]
correct_matches

,index,id,question,response_question,response_answer,response_declarative_sentence,answer,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,is_pretraining,question_category,sentence_category,created_commands,test_answer,test_answer_embedding,response_answer_formatted_embedding,test_answer_cosine_distance,test_answer_correct,test_answer_any_matching_word
53,66,56cfb206234ae51400d9be8f,New Netherland established a permanent Europea...,New Netherland establish –ed a permanent Europ...,1624,New Netherland establish –ed a permanent Europ...,1624,? New-Netherland establish -ed a permanent Eur...,1624,New-Netherland establish -ed a permanent Europ...,False,Wh-Adverbial Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56cfb206234ae51400d9be8f, New-Netherlan...",1624,"[-0.18510516, 0.007434764, 0.02475373, -0.0089...","[-0.18510516, 0.007434764, 0.02475373, -0.0089...",0.0,True,True
56,70,56cedc10aab44d1400b88b1a,What was the regnal name of the Duke of York?,What was the regnal name of the Duke of York?,James II,The regnal name of the Duke of York was James II.,James II,? what was the regnal name of the Duke of York,James-II,the regnal name of the Duke of York was James-II,False,Wh-Object/Complement Question,Subject-Verb-Complement (SVC),"[# ID: 56cedc10aab44d1400b88b1a, the regnal na...",James-II,"[-0.20660716, -0.02500022, 0.020670414, 0.0114...","[-0.20660716, -0.02500022, 0.020670414, 0.0114...",0.0,True,True
65,82,56cedd1caab44d1400b88b42,How many Lenape lived in the area in 1700?,How many Lenape live –d in the area in 1700?,200,200 Lenape live –d in the area in 1700.,200,? how many Lenape live -d in the area in 1700,200,200 Lenape live -d in the area in 1700,False,Quantitative Wh-Question,Subject-Verb-Adverbial (SVA),"[# ID: 56cedd1caab44d1400b88b42, 200 Lenape li...",200,"[-0.17841734, 0.015454305, 0.0016121272, 0.010...","[-0.17841734, 0.015454305, 0.0016121272, 0.010...",0.0,True,True
73,92,56cede40aab44d1400b88b71,In what year was John Peter Zenger tried?,In what year was John Peter Zenger try –ed?,1735,John Peter Zenger was try –ed in 1735.,1735,? in what year was John-Peter-Zenger try -ed,1735,John-Peter-Zenger was try -ed in 1735,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56cede40aab44d1400b88b71, John-Peter-Ze...",1735,"[-0.19004162, -0.008740758, 0.054138605, 0.010...","[-0.19004162, -0.008740758, 0.054138605, 0.010...",0.0,True,True
74,93,56cede40aab44d1400b88b72,On what island did Zenger's trial occur?,On what island did Zenger's trial occur?,Manhattan,Zenger's trial occur –ed on Manhattan.,Manhattan,? on what island did Zengers trial occur,Manhattan,Zengers trial occur -ed on Manhattan,False,Wh-Adverbial Question,Subject-Verb-Adverbial (SVA),"[# ID: 56cede40aab44d1400b88b72, Zengers trial...",Manhattan,"[-0.20684911, -0.003379386, 0.038716637, -0.00...","[-0.20684911, -0.003379386, 0.038716637, -0.00...",0.0,True,True
75,94,56cede40aab44d1400b88b73,In what year was Columbia University chartered?,In what year was Columbia University charter –ed?,1754,Columbia University was charter –ed in 1754.,1754,? in what year was Columbia-University charter...,1754,Columbia-University was charter -ed in 1754,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56cede40aab44d1400b88b73, Columbia-Univ...",1754,"[-0.18632773, -0.0046668183, 0.04927611, 0.002...","[-0.18632773, -0.0046668183, 0.04927611, 0.002...",0.0,True,True
88,109,56cee30faab44d1400b88bf2,Who was the British representative at the Conf...,Who was the British representative at the Conf...,Lord Howe,The British representative at the Conference H...,Lord Howe,? who was the British representative at the Co...,Lord-Howe,the British representative at the Conference-H...,False,Wh-Subject Question,Subject-Verb-Complement (SVC),"[# ID: 56cee30faab44d1400b88bf2, the British r...",Lord-Howe,"[-0.16088775, -0.011601763, 0.017032579, 0.016...","[-0.16088775, -0.011601763, 0.017032579, 0.016...",0.0,True,True
89,111,56

In [18]:
any_matches = response_formatted_df[response_formatted_df["test_answer_any_matching_word"]]
any_matches

,index,id,question,response_question,response_answer,response_declarative_sentence,answer,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,is_pretraining,question_category,sentence_category,created_commands,test_answer,test_answer_embedding,response_answer_formatted_embedding,test_answer_cosine_distance,test_answer_correct,test_answer_any_matching_word
53,66,56cfb206234ae51400d9be8f,New Netherland established a permanent Europea...,New Netherland establish –ed a permanent Europ...,1624,New Netherland establish –ed a permanent Europ...,1624,? New-Netherland establish -ed a permanent Eur...,1624,New-Netherland establish -ed a permanent Europ...,False,Wh-Adverbial Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56cfb206234ae51400d9be8f, New-Netherlan...",1624,"[-0.18510516, 0.007434764, 0.02475373, -0.0089...","[-0.18510516, 0.007434764, 0.02475373, -0.0089...",0.000000,True,True
56,70,56cedc10aab44d1400b88b1a,What was the regnal name of the Duke of York?,What was the regnal name of the Duke of York?,James II,The regnal name of the Duke of York was James II.,James II,? what was the regnal name of the Duke of York,James-II,the regnal name of the Duke of York was James-II,False,Wh-Object/Complement Question,Subject-Verb-Complement (SVC),"[# ID: 56cedc10aab44d1400b88b1a, the regnal na...",James-II,"[-0.20660716, -0.02500022, 0.020670414, 0.0114...","[-0.20660716, -0.02500022, 0.020670414, 0.0114...",0.000000,True,True
65,82,56cedd1caab44d1400b88b42,How many Lenape lived in the area in 1700?,How many Lenape live –d in the area in 1700?,200,200 Lenape live –d in the area in 1700.,200,? how many Lenape live -d in the area in 1700,200,200 Lenape live -d in the area in 1700,False,Quantitative Wh-Question,Subject-Verb-Adverbial (SVA),"[# ID: 56cedd1caab44d1400b88b42, 200 Lenape li...",200,"[-0.17841734, 0.015454305, 0.0016121272, 0.010...","[-0.17841734, 0.015454305, 0.0016121272, 0.010...",0.000000,True,True
73,92,56cede40aab44d1400b88b71,In what year was John Peter Zenger tried?,In what year was John Peter Zenger try –ed?,1735,John Peter Zenger was try –ed in 1735.,1735,? in what year was John-Peter-Zenger try -ed,1735,John-Peter-Zenger was try -ed in 1735,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56cede40aab44d1400b88b71, John-Peter-Ze...",1735,"[-0.19004162, -0.008740758, 0.054138605, 0.010...","[-0.19004162, -0.008740758, 0.054138605, 0.010...",0.000000,True,True
74,93,56cede40aab44d1400b88b72,On what island did Zenger's trial occur?,On what island did Zenger's trial occur?,Manhattan,Zenger's trial occur –ed on Manhattan.,Manhattan,? on what island did Zengers trial occur,Manhattan,Zengers trial occur -ed on Manhattan,False,Wh-Adverbial Question,Subject-Verb-Adverbial (SVA),"[# ID: 56cede40aab44d1400b88b72, Zengers trial...",Manhattan,"[-0.20684911, -0.003379386, 0.038716637, -0.00...","[-0.20684911, -0.003379386, 0.038716637, -0.00...",0.000000,True,True
75,94,56cede40aab44d1400b88b73,In what year was Columbia University chartered?,In what year was Columbia University charter –ed?,1754,Columbia University was charter –ed in 1754.,1754,? in what year was Columbia-University charter...,1754,Columbia-University was charter -ed in 1754,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56cede40aab44d1400b88b73, Columbia-Univ...",1754,"[-0.18632773, -0.0046668183, 0.04927611, 0.002...","[-0.18632773, -0.0046668183, 0.04927611, 0.002...",0.000000,True,True
77,96,56cede40aab44d1400b88b75,What was the original name of Columbia Univers...,What was the origin –al name of Columbia Unive...,King's College,The origin –al name of Columbia University was...,King's College,? what was the origin -al name of Columbia-Uni...,Kings-College,the origin -al name of Columbia-University was...,False,Wh-Object/Complement Question,Subject-Verb-Complement (SVC),"[# ID: 56cede40aab44d1400b88b75, the origin -a...",was Kings-College,"[-0.1823988, -0.017900608, 0.014207478, -0.001...","[-0.19089682,

In [19]:
incorrect_matches = response_formatted_df[~response_formatted_df["test_answer_correct"]]
incorrect_matches

,index,id,question,response_question,response_answer,response_declarative_sentence,answer,response_question_formatted,response_answer_formatted,response_declarative_sentence_formatted,is_pretraining,question_category,sentence_category,created_commands,test_answer,test_answer_embedding,response_answer_formatted_embedding,test_answer_cosine_distance,test_answer_correct,test_answer_any_matching_word
0,0,56ce304daab44d1400b8850e,What city in the United States has the highest...,What city in the United States has the high –e...,New York,The city in the United States with the high –e...,New York,? what city in the United-States has the high ...,New-York,the city in the United-States with the high -e...,False,Wh-Subject Question,Subject-Verb-Complement (SVC),"[# ID: 56ce304daab44d1400b8850e, the city in t...",1700s,"[-0.18142706, -0.001205836, 0.046104997, 0.007...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.208811,False,False
1,1,56ce304daab44d1400b8850f,In what city is the United Nations based?,In what city is the United Nations base –d?,New York,The United Nations is base –d in New York.,New York,? in what city is the United-Nations base -d,New-York,the United-Nations is base -d in New-York,False,Wh-Adverbial Question,Passive Construction,"[# ID: 56ce304daab44d1400b8850f, the United-Na...",Exploitation number of updates >= 4000,"[-0.15511338, 0.028740732, 0.008751592, 0.0579...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.306653,False,False
2,2,56ce304daab44d1400b88510,What city has been called the cultural capital...,What city has been call –ed the culture –al ca...,New York,New York has been call –ed the culture –al cap...,New York,? what city has been call -ed the culture -al ...,New-York,New-York has been call -ed the culture -al cap...,False,Wh-Subject Question,Passive Construction,"[# ID: 56ce304daab44d1400b88510, New-York has ...",One-World-Trade-Center,"[-0.16647865, 0.01233991, 0.051709116, 0.00260...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.207379,False,False
3,3,56ce304daab44d1400b88511,What American city welcomes the largest number...,What American city welcome –s the large –st nu...,New York,New York is the American city that welcome –s ...,New York,? what American city welcome -s the large -st ...,New-York,New-York is the American city that welcome -s ...,False,Wh-Subject Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56ce304daab44d1400b88511, New-York is t...",Exploitation number of updates >= 4000,"[-0.15511338, 0.028740732, 0.008751592, 0.0579...","[-0.1972834, 0.00014738683, 0.026876723, 0.000...",0.306653,False,False
4,6,56ce3124aab44d1400b8852a,How many boroughs comprise New York City?,How many borough –s comprise New York City?,five,New York City is comprise –d of five borough –s.,five,? how many borough -s comprise New-York-City,five,New-York-City is comprise -d of five borough -s,False,Quantitative Wh-Question,Passive Construction,"[# ID: 56ce3124aab44d1400b8852a, New-York-City...",NO ANSWER GIVEN,"[-0.17646857, 0.017760377, 0.028869705, 0.0407...","[-0.21607131, 0.0066142594, 0.021365147, -0.00...",0.146334,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,723,56d1204617492d1400aab9fc,What federal district court has jurisdiction o...,What federal district court has jurisdiction o...,the District Court for the Eastern District of...,The District Court for the Eastern District of...,the District Court for the Eastern District of...,? what federal district court has jurisdiction...,the District-Court for the Eastern-District of...,the District-Court for the Eastern-District of...,False,Wh-Subject Question,Subject-Verb-Object (SVO/SVOA),"[# ID: 56d1204617492d1400aab9fc, the District-...",NO ANSWER GIVEN,"[-0.17646857, 0.017760377, 0.028869705, 0.0407...","[-0.1400723, 0.008040977, 0.020870337, 0.00184...",0.356941,False,False
597,724,56d1204617492d1400aab9fd,In what borough is the New York City Hall found?,In what borough is t

In [20]:
#write the results to a file and export the results dataframe to a tsv file
tsv_results_filename = "test_nyc_results" + datetime.datetime.now().strftime("_%Y%m%d_%H%M%S") + ".tsv"
tsv_results_filepath = os.path.join(test_results_dir, tsv_results_filename)
response_formatted_df.to_csv(tsv_results_filepath, sep="\t", index=False)

results_summary_filename = "test_nyc_results_summary" + datetime.datetime.now().strftime("_%Y%m%d_%H%M%S") + ".txt"
results_summary_filepath = os.path.join(test_results_dir, results_summary_filename)

with open(results_summary_filepath, 'w') as results_file:
    #write the number of samples tested
    results_file.write(f"total number of samples\t{len(test_input_lines) / 2}\n")
    results_file.write(f"number_of_test_answers\t{len(response_formatted_df)}\n")
    results_file.write(f"percentage_correct\t{percentage_correct}\n")
    results_file.write(f"percentage_any_word_matches\t{percentage_any_word_matches}\n")
    results_file.write(f"number of test answers longer than 20 words (removed)\t{num_long_answers}\n")
    # write the rows that had exact word matches to the file
    results_file.write("\nRows with exact matches:\n")
    results_file.write(
        correct_matches[["response_question", "response_answer", "test_answer"]].to_markdown(index=False))
    #write the rows in any_matches to the file
    results_file.write("\nRows with any word matches:\n")
    results_file.write(any_matches[["response_question", "response_answer", "test_answer"]].to_markdown(index=False))
    # write the rows that had a close cosine distance to the file
    results_file.write(f"\nRows with cosine distance less than {cosine_distance_threshold}:\n")
    results_file.write(
        close_cosine_distance_df[
            ["response_question", "response_answer", "test_answer", "test_answer_cosine_distance"]].to_markdown(
            index=False))
    # write the rows that had a close cosine distance and any word match to the file
    results_file.write(f"\nRows with cosine distance less than {cosine_distance_threshold} and any word match:\n")
    results_file.write(close_cosine_distance_correct_df[
                           ["response_question", "response_answer", "test_answer",
                            "test_answer_cosine_distance"]].to_markdown(index=False))
    #write the rows that had any matches and with a close cosine distance to the file
    results_file.write(f"\nRows with cosine distance less than {cosine_distance_threshold} and exact match:\n")
    results_file.write(correct_matches[
                           ["response_question", "response_answer", "test_answer",
                            "test_answer_cosine_distance"]].to_markdown(index=False))
print(f"results written to {tsv_results_filepath} and {results_summary_filepath}")

results written to /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/12/results/test_nyc_results_20251015_173148.tsv and /Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/experiments/12/results/test_nyc_results_summary_20251015_173149.txt
